In [ ]:
import os

print(os.getcwd())
import os
os.chdir(r"C:\Users\Farha Tarique\Desktop\BIOINFORMATICS") # set directory
print(os.getcwd())  


c:\Users\Farha Tarique\AppData\Local\Programs\Microsoft VS Code
C:\Users\Farha Tarique\Desktop\BIOINFORMATICS


In [7]:
import requests
import csv

protein_ids = ["P01308", "P04637", "Q9Y261", "P69905"]

with open("proteins_extended.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Entry ID", "Protein Name", "Gene Names", "Organism",
        "Length", "Mass", "Function", "Subcellular Location", "Keywords", "Sequence"
    ])

    for entry_id in protein_ids:
        url = f"https://rest.uniprot.org/uniprotkb/{entry_id}.json"
        r = requests.get(url)
        if r.status_code != 200:
            print(f"Failed: {entry_id}")
            continue

        data = r.json()

        # Sequence info
        seq_info = data.get("sequence", {})
        sequence = seq_info.get("value", "")
        mass = seq_info.get("mass", "")
        length = seq_info.get("length", "")

        # Protein name
        protein_name = data.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value", "")

        # Gene names (may be multiple)
        genes = data.get("genes", [])
        gene_names = ", ".join([g.get("geneName", {}).get("value", "") for g in genes])

        # Organism
        organism = data.get("organism", {}).get("scientificName", "")

        # Function
        function = "N/A"
        for c in data.get("comments", []):
            if c.get("commentType") == "FUNCTION":
                function = c["texts"][0]["value"]
                break

        # Subcellular location
        subcellular = []
        for c in data.get("comments", []):
            if c.get("commentType") == "SUBCELLULAR LOCATION":
                for loc in c.get("subcellularLocations", []):
                    loc_text = loc.get("location", {}).get("value", "")
                    if loc_text:
                        subcellular.append(loc_text)
        subcellular_location = ", ".join(subcellular) if subcellular else "N/A"

        # Keywords
        keywords = [k.get("value", "") for k in data.get("keywords", [])]
        keywords_str = ", ".join(keywords)

        writer.writerow([
            entry_id, protein_name, gene_names, organism,
            length, mass, function, subcellular_location, keywords_str, sequence
        ])

        print(f"✅ Fetched {entry_id}")


✅ Fetched P01308
✅ Fetched P04637
✅ Fetched Q9Y261
✅ Fetched P69905


In [9]:
import pandas as pd

df = pd.read_csv("proteins_extended.csv")
print(df.head())  # first 5 rows


  Entry ID                      Protein Name  Gene Names      Organism  \
0   P01308                           Insulin         INS  Homo sapiens   
1   P04637        Cellular tumor antigen p53        TP53  Homo sapiens   
2   Q9Y261  Hepatocyte nuclear factor 3-beta       FOXA2  Homo sapiens   
3   P69905          Hemoglobin subunit alpha  HBA1, HBA2  Homo sapiens   

   Length  Mass                                           Function  \
0     110   NaN  Insulin decreases blood glucose concentration....   
1     393   NaN  Multifunctional transcription factor that indu...   
2     457   NaN  Transcription factor that is involved in embry...   
3     142   NaN  Involved in oxygen transport from the lung to ...   

                                Subcellular Location  \
0                                           Secreted   
1  Cytoplasm, Nucleus, Nucleus, PML body, Endopla...   
2                                 Nucleus, Cytoplasm   
3                                                NaN  